# Boilerplate

In [1]:
import sys
sys.path.insert(0, 'venv/lib/python3.10/site-packages')
import asyncio
import time

from nwv2_python_wrapper import *
import nwv2_python_wrapper
init_logging()

# Run

List all Python wrappers

In [2]:
[f for f in dir(nwv2_python_wrapper) if not f.startswith('__') and f != 'nwv2_python_wrapper']

['BroadcastChatMessageW',
 'EndpointW',
 'FilterCmdW',
 'FilterInterface',
 'FilterModeW',
 'FilterNoticeW',
 'FilterRspW',
 'GameOptionsW',
 'GameOutcomeW',
 'GameUpdateW',
 'GenPartInfoW',
 'GenStateDiffPartW',
 'GenStateDiffW',
 'NetRegionW',
 'PacketSettingsW',
 'PacketW',
 'PlayerInfoW',
 'ProcessUniqueIdW',
 'RequestActionW',
 'ResponseCodeW',
 'RoomListW',
 'TransportCmdW',
 'TransportInterface',
 'TransportNoticeW',
 'TransportRspW',
 'UniUpdateW',
 'debug_hello',
 'init_logging',
 'new_transport_interface']

## Transport layer setup

Create a Client and Server Transport layer and run each in the background.

In [3]:
SERVER_PORT = 62013

In [4]:
client_tiface_inner = await new_transport_interface("0.0.0.0", 0)
client_tiface_inner.run()

server_tiface_inner = await new_transport_interface("0.0.0.0", SERVER_PORT)
server_tiface_inner.run()

Mon 2023-01-09 20:39:40.865600 [TRACE] - registering event source with poller: token=Token(0), interests=READABLE
Mon 2023-01-09 20:39:40.866310 [INFO ] - [TRANSPORT] Attempting to bind to 0.0.0.0:0
Mon 2023-01-09 20:39:40.866341 [TRACE] - registering event source with poller: token=Token(1), interests=READABLE | WRITABLE
Mon 2023-01-09 20:39:40.866908 [INFO ] - [TRANSPORT] Attempting to bind to 0.0.0.0:62013
Mon 2023-01-09 20:39:40.906674 [TRACE] - registering event source with poller: token=Token(2), interests=READABLE | WRITABLE


<Future pending cb=[<builtins.PyDoneCallback object at 0x7fa6cc5a6970>()]>

## Wrap the PYO3 Transport Interface in a Python class with the same methods

Note that there is no `run()`. Hence, the passed in `TransportInterface` instance must have already had its `run()` method called.

In [ ]:
CRESET    = '\33[0m'
CBOLD     = '\33[1m'
CITALIC   = '\33[3m'
CURL      = '\33[4m'
CBLINK    = '\33[5m'
CBLINK2   = '\33[6m'
CSELECTED = '\33[7m'

CBLACK  = '\33[30m'
CRED    = '\33[31m'
CGREEN  = '\33[32m'
CYELLOW = '\33[33m'
CBLUE   = '\33[34m'
CVIOLET = '\33[35m'
CBEIGE  = '\33[36m'
CWHITE  = '\33[37m'

def cprint(tag, message):
    if tag == "client":
        color = CYELLOW
    else:
        color = CVIOLET
    print(f"{color}{message}{CRESET}")
        

class LoggingTransportInterface:
    def __init__(self, inner, tag):
        self.inner = inner
        self.ctr = 0
        self.skipped_count = None
        self.tag = tag
        
    async def command_response(self, transport_cmd):
        cprint(self.tag, f"LTI({self.tag}){self.ctr}{self.skipped()}: received transport_cmd {transport_cmd}")
        self.ctr += 1
        transport_rsp = await self.inner.command_response(transport_cmd)
        cprint(self.tag, f"LTI({self.tag}){self.ctr}: got transport_rsp back from Transport layer: {transport_rsp}")
        self.ctr += 1
        return transport_rsp
    
    def get_notifications(self):
        transport_notif_list = self.inner.get_notifications()
        if self.skipped_count is None:
            cprint(self.tag, f"LTI({self.tag}){self.ctr}: got transport notifications: {transport_notif_list}")
            self.skipped_count = 0
            if len(transport_notif_list) > 0:
                self.reset()
        else:
            self.skipped_count += 1
        self.ctr += 1
        return transport_notif_list
    
    def skipped(self):
        if self.skipped_count is None:
            return ""
        count = self.skipped_count
        self.skipped_count = None
        return f" skipped {count}"
    
    def reset(self):
        self.skipped_count = None
        
client_tiface = LoggingTransportInterface(client_tiface_inner, "client")
client_tiface

server_tiface = LoggingTransportInterface(server_tiface_inner, "server")
server_tiface

# Filter layer setup

In [6]:
client_fiface = FilterInterface(client_tiface, FilterModeW("client"))
server_fiface = FilterInterface(server_tiface, FilterModeW("server"))

(client_fiface, server_fiface)

(<FilterInterface at 0x7fa6cc654f40>, <FilterInterface at 0x7fa6cc654b80>)

Find methods to run

In [7]:
[m for m in dir(client_fiface) if not m.startswith('__')]

['command_response', 'get_notifications', 'notif_poll_ms', 'run']

Run Client and Server Filter!

In [8]:
client_fiface_fut = client_fiface.run()
server_fiface_fut = server_fiface.run()

# After waiting a bit, the above future should not have completed
time.sleep(0.1)
assert not client_fiface_fut.done() and not server_fiface_fut.done()

LTI(client)0: got transport notifications: []
LTI(server)0: got transport notifications: []


In [9]:
client_fiface.notif_poll_ms = 100


## Filter layer operation

### As a client, connect to server

In [10]:
import random
id = random.randint(1,10000)
connect_action = RequestActionW("connect", name="FilterNotebookClient{}".format(id), client_version="0.0.666")
connect_action

Connect { name: "FilterNotebookClient6224", client_version: "0.0.666" }

In [11]:
jupyter_endpoint = EndpointW(f"127.0.0.1:{SERVER_PORT}")
filter_cmd = FilterCmdW("sendrequestaction", endpoint=jupyter_endpoint, action=connect_action)
filter_cmd

SendRequestAction { endpoint: Endpoint(127.0.0.1:62013), action: Connect { name: "FilterNotebookClient6224", client_version: "0.0.666" } }

In [12]:
await client_fiface.command_response(filter_cmd)

Mon 2023-01-09 20:39:41.087944 [TRACE] - [FILTER] New command: SendRequestAction { endpoint: Endpoint(127.0.0.1:62013), action: Connect { name: "FilterNotebookClient6224", client_version: "0.0.666" } }
LTI(client)5 skipped 4: received transport_cmd NewEndpoint { endpoint: Endpoint(127.0.0.1:62013), timeout: 250ms }
Mon 2023-01-09 20:39:41.088234 [TRACE] - [TRANSPORT] Filter Request: NewEndpoint { endpoint: Endpoint(127.0.0.1:62013), timeout: 250ms }
LTI(client)6: got transport_rsp back from Transport layer: TransportRsp::Accepted
LTI(client)7: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 0 }, retry_interval: 250ms }], packets: [Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient6224", client_version: "0.0.666" } }] }
LTI(client)8: got transport_rsp back from Transport layer: TransportRsp::Accepted
Mon 2023-01-09 20:39:41.088465 [TRACE

Accepted

Mon 2023-01-09 20:39:41.173501 [TRACE] - [TRANSPORT] Filter Request: DropPacket { endpoint: Endpoint(127.0.0.1:62013), tid: ProcessUniqueId { prefix: 0, offset: 0 } }


In [13]:
client_notifications = client_fiface.get_notifications()
assert len(client_notifications) > 0
new_response_code = client_notifications[0]
new_response_code

NewResponseCode { endpoint: Endpoint(127.0.0.1:62013), code: OK }

In [14]:
cookie = new_response_code.response_code.cookie
cookie

In [15]:
cookie

In [16]:
server_notifications = server_fiface.get_notifications()
assert len(server_notifications) > 0
server_notifications[0]

thread '<unnamed>' panicked at 'failed to acquire notify rx lock: TryLockError(())', nwv2-python-wrapper/src/filter/mod.rs:212:45
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace


PanicException: failed to acquire notify rx lock: TryLockError(())

### Convenience function

In [ ]:
async def request_action_to_filter(req_action, endpoint=chococat_endpoint):
    filter_cmd = FilterCmdW("sendrequestaction", endpoint=endpoint, action=req_action)
    return await client_fiface.command_response(filter_cmd)

### Join a room

In [ ]:
await request_action_to_filter(RequestActionW("joinroom", room_name="general"))

Due to a bug in the Netwayste v1 server, there is a `BadRequest { error_msg: "cannot join game because in-game" }` filter notice following the `JoinedRoom` filter notice, below. Apparently, it's caused by a packet retry of the `JoinRoom` request action by the Transport layer, which is erroneously being treated as a second request action by the NWv1 server. It's erroneous because the request sequence is the same, so it should have been dropped by the server.

In [ ]:
client_fiface.get_notifications()

LTI(client)11: got transport_rsp back from Transport layer: TransportRsp::Accepted
Mon 2023-01-09 20:39:41.235225 [TRACE] - [FILTER] Transport Command Accepted
LTI(client)12: got transport notifications: [TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }]
LTI(client)13: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 1 }, retry_interval: 250ms }], packets: [Request { sequence: 2, response_ack: Some(1), cookie: None, action: KeepAlive { latest_response_ack: 1 } }] }
Mon 2023-01-09 20:39:41.275142 [TRACE] - [TRANSPORT] Filter Request: SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 1 }, retry_interval: 250ms }], packets: [Request { sequence: 2, response_ack: Some(1), cookie: None, action: KeepAlive { latest_response_ack: 1 } }] }
LTI(client)14: got transport_rsp back from Transport layer: Trans

LTI(server)39: got transport notifications: []
LTI(client)26: got transport notifications: [TransportNotice::PacketDelivery { endpoint: Endpoint(127.0.0.1:62013), packet: Response { sequence: 4, request_ack: Some(4), code: OK } }]
Mon 2023-01-09 20:39:41.680607 [INFO ] - [FILTER] Packet Taken from Endpoint Endpoint(127.0.0.1:62013).
Mon 2023-01-09 20:39:41.680621 [TRACE] - [FILTER] Took packet: Response { sequence: 4, request_ack: Some(4), code: OK }
LTI(client)27: received transport_cmd DropPacket { endpoint: Endpoint(127.0.0.1:62013), tid: ProcessUniqueId { prefix: 2, offset: 2 } }
Mon 2023-01-09 20:39:41.681050 [TRACE] - [TRANSPORT] Filter Request: DropPacket { endpoint: Endpoint(127.0.0.1:62013), tid: ProcessUniqueId { prefix: 2, offset: 2 } }
LTI(client)28: got transport_rsp back from Transport layer: TransportRsp::Accepted
Mon 2023-01-09 20:39:41.681334 [TRACE] - [FILTER] Transport Command Accepted
LTI(client)29: got transport notifications: [TransportNotice::EndpointIdle { endpo

LTI(client)43: got transport notifications: [TransportNotice::PacketDelivery { endpoint: Endpoint(127.0.0.1:62013), packet: Response { sequence: 7, request_ack: Some(7), code: OK } }]
Mon 2023-01-09 20:39:42.187416 [INFO ] - [FILTER] Packet Taken from Endpoint Endpoint(127.0.0.1:62013).
Mon 2023-01-09 20:39:42.187428 [TRACE] - [FILTER] Took packet: Response { sequence: 7, request_ack: Some(7), code: OK }
LTI(client)44: received transport_cmd DropPacket { endpoint: Endpoint(127.0.0.1:62013), tid: ProcessUniqueId { prefix: 1, offset: 3 } }
Mon 2023-01-09 20:39:42.187794 [TRACE] - [TRANSPORT] Filter Request: DropPacket { endpoint: Endpoint(127.0.0.1:62013), tid: ProcessUniqueId { prefix: 1, offset: 3 } }
LTI(client)45: got transport_rsp back from Transport layer: TransportRsp::Accepted
Mon 2023-01-09 20:39:42.188207 [TRACE] - [FILTER] Transport Command Accepted
LTI(client)46: got transport notifications: [TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }]
LTI(client)47

LTI(client)60: got transport notifications: [TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::PacketDelivery { endpoint: Endpoint(127.0.0.1:62013), packet: Response { sequence: 10, request_ack: Some(10), code: OK } }]
Mon 2023-01-09 20:39:42.692302 [INFO ] - [FILTER] Packet Taken from Endpoint Endpoint(127.0.0.1:62013).
Mon 2023-01-09 20:39:42.692315 [TRACE] - [FILTER] Took packet: Response { sequence: 10, request_ack: Some(10), code: OK }
LTI(client)61: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 2 }, retry_interval: 250ms }], packets: [Request { sequence: 11, response_ack: Some(9), cookie: None, action: KeepAlive { latest_response_ack: 9 } }] }
Mon 2023-01-09 20:39:42.692781 [TRACE] - [TRANSPORT] Filter Request: SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 2 }, ret

LTI(client)73: got transport notifications: [TransportNotice::PacketDelivery { endpoint: Endpoint(127.0.0.1:62013), packet: Response { sequence: 12, request_ack: Some(13), code: OK } }, TransportNotice::PacketDelivery { endpoint: Endpoint(127.0.0.1:62013), packet: Response { sequence: 13, request_ack: Some(13), code: OK } }]
Mon 2023-01-09 20:39:42.996577 [INFO ] - [FILTER] Packet Taken from Endpoint Endpoint(127.0.0.1:62013).
Mon 2023-01-09 20:39:42.996590 [TRACE] - [FILTER] Took packet: Response { sequence: 12, request_ack: Some(13), code: OK }
LTI(client)74: received transport_cmd DropPacket { endpoint: Endpoint(127.0.0.1:62013), tid: ProcessUniqueId { prefix: 1, offset: 6 } }
Mon 2023-01-09 20:39:42.996602 [INFO ] - [FILTER] Packet Taken from Endpoint Endpoint(127.0.0.1:62013).
Mon 2023-01-09 20:39:42.996606 [TRACE] - [FILTER] Took packet: Response { sequence: 13, request_ack: Some(13), code: OK }
Mon 2023-01-09 20:39:42.997076 [TRACE] - [TRANSPORT] Filter Request: DropPacket { end

...

LTI(client)295: got transport notifications: [TransportNotice::PacketDelivery { endpoint: Endpoint(127.0.0.1:62013), packet: Response { sequence: 54, request_ack: Some(54), code: OK } }]
Mon 2023-01-09 20:39:48.670610 [INFO ] - [FILTER] Packet Taken from Endpoint Endpoint(127.0.0.1:62013).
Mon 2023-01-09 20:39:48.670621 [TRACE] - [FILTER] Took packet: Response { sequence: 54, request_ack: Some(54), code: OK }
LTI(client)296: received transport_cmd DropPacket { endpoint: Endpoint(127.0.0.1:62013), tid: ProcessUniqueId { prefix: 3, offset: 19 } }
Mon 2023-01-09 20:39:48.670981 [TRACE] - [TRANSPORT] Filter Request: DropPacket { endpoint: Endpoint(127.0.0.1:62013), tid: ProcessUniqueId { prefix: 3, offset: 19 } }
LTI(client)297: got transport_rsp back from Transport layer: TransportRsp::Accepted
Mon 2023-01-09 20:39:48.671367 [TRACE] - [FILTER] Transport Command Accepted
LTI(client)298: got transport notifications: [TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }]
LTI

LTI(client)312: got transport notifications: [TransportNotice::PacketDelivery { endpoint: Endpoint(127.0.0.1:62013), packet: Response { sequence: 57, request_ack: Some(57), code: OK } }]
Mon 2023-01-09 20:39:49.178017 [INFO ] - [FILTER] Packet Taken from Endpoint Endpoint(127.0.0.1:62013).
Mon 2023-01-09 20:39:49.178028 [TRACE] - [FILTER] Took packet: Response { sequence: 57, request_ack: Some(57), code: OK }
LTI(client)313: received transport_cmd DropPacket { endpoint: Endpoint(127.0.0.1:62013), tid: ProcessUniqueId { prefix: 1, offset: 42 } }
Mon 2023-01-09 20:39:49.178312 [TRACE] - [TRANSPORT] Filter Request: DropPacket { endpoint: Endpoint(127.0.0.1:62013), tid: ProcessUniqueId { prefix: 1, offset: 42 } }
LTI(client)314: got transport_rsp back from Transport layer: TransportRsp::Accepted
Mon 2023-01-09 20:39:49.178611 [TRACE] - [FILTER] Transport Command Accepted
LTI(client)315: got transport notifications: [TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }]
LTI

LTI(client)331: got transport notifications: [TransportNotice::PacketDelivery { endpoint: Endpoint(127.0.0.1:62013), packet: Response { sequence: 60, request_ack: Some(61), code: OK } }, TransportNotice::PacketDelivery { endpoint: Endpoint(127.0.0.1:62013), packet: Response { sequence: 61, request_ack: Some(61), code: OK } }]
Mon 2023-01-09 20:39:49.684230 [INFO ] - [FILTER] Packet Taken from Endpoint Endpoint(127.0.0.1:62013).
Mon 2023-01-09 20:39:49.684242 [TRACE] - [FILTER] Took packet: Response { sequence: 60, request_ack: Some(61), code: OK }
Mon 2023-01-09 20:39:49.684252 [INFO ] - [FILTER] Packet Taken from Endpoint Endpoint(127.0.0.1:62013).
Mon 2023-01-09 20:39:49.684255 [TRACE] - [FILTER] Took packet: Response { sequence: 61, request_ack: Some(61), code: OK }
LTI(client)332: received transport_cmd DropPacket { endpoint: Endpoint(127.0.0.1:62013), tid: ProcessUniqueId { prefix: 2, offset: 48 } }
Mon 2023-01-09 20:39:49.684617 [TRACE] - [TRANSPORT] Filter Request: DropPacket { 

LTI(client)344: got transport notifications: [TransportNotice::PacketDelivery { endpoint: Endpoint(127.0.0.1:62013), packet: Response { sequence: 63, request_ack: Some(63), code: OK } }]
Mon 2023-01-09 20:39:49.987508 [INFO ] - [FILTER] Packet Taken from Endpoint Endpoint(127.0.0.1:62013).
Mon 2023-01-09 20:39:49.987521 [TRACE] - [FILTER] Took packet: Response { sequence: 63, request_ack: Some(63), code: OK }
LTI(client)345: received transport_cmd DropPacket { endpoint: Endpoint(127.0.0.1:62013), tid: ProcessUniqueId { prefix: 2, offset: 53 } }
Mon 2023-01-09 20:39:49.987988 [TRACE] - [TRANSPORT] Filter Request: DropPacket { endpoint: Endpoint(127.0.0.1:62013), tid: ProcessUniqueId { prefix: 2, offset: 53 } }
LTI(client)346: got transport_rsp back from Transport layer: TransportRsp::Accepted
Mon 2023-01-09 20:39:49.988295 [TRACE] - [FILTER] Transport Command Accepted
LTI(client)347: got transport notifications: [TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }]
LTI

LTI(client)358: got transport notifications: [TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::PacketDelivery { endpoint: Endpoint(127.0.0.1:62013), packet: Response { sequence: 64, request_ack: Some(64), code: OK } }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }]
Mon 2023-01-09 20:39:50.392555 [INFO ] - [FILTER] Packet Taken from Endpoint Endpoint(127.0.0.1:62013).
Mon 2023-01-09 20:39:50.392569 [TRACE] - [FILTER] Took packet: Response { sequence: 64, request_ack: Some(64), code: OK }
LTI(client)359: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 3, offset: 24 }, retry_interval: 250ms }], packets: [Request { sequence: 66, response_ack: Some(65), cookie: None, action: KeepAlive { latest_response_ack: 65 } }] }
Mon 2023-01-09 20:39:50.392972 [TRACE] - [TRANSPORT] Filter Request: SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet